In [2]:
'''
Resource to rebuild https://stackabuse.com/accessing-the-twitter-api-with-python/
'''


'\nResource to rebuild https://stackabuse.com/accessing-the-twitter-api-with-python/\n'

In [ ]:
import pandas as pd
import numpy as np
import json
import twython as twitter


# Enter your keys/secrets as strings in the following fields
credentials = {}
credentials['CONSUMER_KEY'] = ...
credentials['CONSUMER_SECRET'] = ...
credentials['ACCESS_TOKEN'] = ...
credentials['ACCESS_SECRET'] = ...


# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)